# Bibliotecas:

In [ ]:
import pandas as pd
import numpy as np
import math
pd.options.mode.chained_assignment = None

Obtendo o Banco de Dados da COVID-19 no Brasil com informações de número de casos e mortos em cada cidade.

# Montagem de Drive:

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
path = "/content/drive/MyDrive/Colab Notebooks"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Leitura dos Bancos de Dados:

In [ ]:
dadosCovid = pd.read_csv(path+"/caso_full.csv.gz",index_col=False,header=None,squeeze=True,low_memory=False)
#Origem dos dados: https://brasil.io/dataset/covid19/files/

#Subset das informações mais relevantes sobre a vacinação.
dadosVacinacao = pd.read_csv(path+"/dadosVacinacao.zip",index_col=False,header=None,squeeze=True,low_memory=False)
#Fonte: https://coronavirus.saude.mg.gov.br/dadosabertos -  Sistemas Vacinação

In [ ]:
display(dadosCovid.head(6))
display(dadosVacinacao.head(2))

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
0,city,city_ibge_code,date,epidemiological_week,estimated_population,estimated_population_2019,is_last,is_repeated,last_available_confirmed,last_available_confirmed_per_100k_inhabitants,last_available_date,last_available_death_rate,last_available_deaths,order_for_place,place_type,state,new_confirmed,new_deaths
1,Rio Branco,1200401,2020-03-17,202012,413418,407319,False,False,3,0.72566,2020-03-17,0.0000,0,1,city,AC,3,0
2,NaN,12,2020-03-17,202012,894470,881935,False,False,3,0.33539,2020-03-17,0.0000,0,1,state,AC,3,0
3,Rio Branco,1200401,2020-03-18,202012,413418,407319,False,False,3,0.72566,2020-03-18,0.0000,0,2,city,AC,0,0
4,NaN,12,2020-03-18,202012,894470,881935,False,False,3,0.33539,2020-03-18,0.0000,0,2,state,AC,0,0
5,Rio Branco,1200401,2020-03-19,202012,413418,407319,False,False,4,0.96754,2020-03-19,0.0000,0,3,city,AC,1,0


,0,1,2,3,4,5,6,7
0,ID,faixa_etaria,sexo,municipio_res,macro,vacina_nome,data_aplicacao,dose
1,1,60 a 69 anos,MASCULINO,BELO HORIZONTE,CENTRO,CORONAVAC,2021-05-20,2ª Dose


# Parte 2 - *Data Prep*

## **1)** Como nossa análise se restringirá a MG, vamos selecionar apenas as informações dos municípios da referida UF.

In [ ]:
dadosMG = dadosCovid[dadosCovid[15] == "MG"]
display(dadosMG)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
627858,Divinópolis,3122306,2020-03-08,202011,240408,238230,False,False,1,0.41596,2020-03-08,0.0000,0,1,city,MG,1,0
627859,NaN,31,2020-03-08,202011,21292666,21168791,False,False,1,0.00470,2020-03-08,0.0000,0,1,state,MG,1,0
627860,Divinópolis,3122306,2020-03-09,202011,240408,238230,False,False,1,0.41596,2020-03-09,0.0000,0,2,city,MG,0,0
627861,NaN,31,2020-03-09,202011,21292666,21168791,False,False,1,0.00470,2020-03-09,0.0000,0,2,state,MG,0,0
627862,Divinópolis,3122306,2020-03-10,202011,240408,238230,False,False,1,0.41596,2020-03-10,0.0000,0,3,city,MG,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1005150,Água Boa,3100609,2021-08-08,202132,13523,13735,False,True,511,3778.74732,2021-06-18,0.0137,7,408,city,MG,0,0
1005151,Água Comprida,3100708,2021-08-08,202132,1992,1999,False,True,169,8483.93574,2021-06-18,0.0473,8,436,city,MG,0,0
1005152,Águas Formosas,3100906,2021-08-08,202132,19247,19207,False,True,899,4670.85780,2021-06-18,0.0345,31,431,city,MG,0,0
1005153,Águas Vermelhas,3101003,2021-08-08,202132,13599,13539,False,True,402,2956.09971,2021-06-18,0.0174,7,446,city,MG,0,0


## **2)** Removendo os NaN's e nulos.

In [ ]:
# dadosMG = dadosMG.dropna()
print(len(dadosMG[dadosMG[14] == 'state'])) #dropar os 'state' e padronificar os NaN
teste = dadosMG.isnull()
teste2 = teste.any(axis=1)
teste3 = dadosMG[teste2]
display(teste3)

In [ ]:
print(f"Quantidade de NA na base de casos do covid: {dadosMG.isnull().sum().sum()}")

for i in range(18):
  print(f"Quantidade de NA na base de casos do covid {i}: {dadosMG[i].isnull().sum().sum()}")

In [ ]:
for i in range(18):
  dadosMG[i] = dadosMG[i].str.strip()

In [ ]:
identificacoes = ['NÃO ESPECIFICADO', 'NÃO IDENTIFICADO', 'NÃO LOCALIZADO']
# dadosVacinacao[1] = dadosVacinacao[1].fillna('NÃO ESPECIFICADO')
# dadosVacinacao[dadosVacinacao[1] == 'NÃO ESPECIFICADO']
# dadosVacinacao[2] = dadosVacinacao[2].fillna('NÃO IDENTIFICADO')
# dadosVacinacao[dadosVacinacao[2] == 'NÃO IDENTIFICADO']
# dadosVacinacao[3] = dadosVacinacao[3].fillna('NÃO LOCALIZADO')
# dadosVacinacao[dadosVacinacao[3] == 'NÃO LOCALIZADO']
for i in range(3):
  dadosVacinacao[i+1] = dadosVacinacao[i+1].fillna(identificacoes[i])
dadosVacinacao[dadosVacinacao[3] == 'NÃO LOCALIZADO']



In [ ]:
print(f"Quantidade de NA na vacinação: {dadosVacinacao.isna().sum().sum()}")

for i in range(8):
  print(f"Quantidade de NA na vacinação coluna {i}: {dadosVacinacao[i].isnull().sum().sum()}")
 


Quantidade de NA na vacinação: 0
Quantidade de NA na vacinação coluna 0: 0
Quantidade de NA na vacinação coluna 1: 0
Quantidade de NA na vacinação coluna 2: 0
Quantidade de NA na vacinação coluna 3: 0
Quantidade de NA na vacinação coluna 4: 0
Quantidade de NA na vacinação coluna 5: 0
Quantidade de NA na vacinação coluna 6: 0
Quantidade de NA na vacinação coluna 7: 0


## **3)** Alteração de tipos de alguns atributos:

In [ ]:
"""
  Como nós iremos fazer a soma de casos e mortes, esses valores precisam ser numéricos.
"""
dadosMG[16] = pd.to_numeric(dadosMG[16])
dadosMG[17] = pd.to_numeric(dadosMG[17])
display(dadosMG)
#Fonte: https://www.ti-enxame.com/pt/python/alterar-tipo-de-dados-de-colunas-em-pandas/1072113742/

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
627858,Divinópolis,3122306,2020-03-08,202011,240408,238230,False,False,1,0.41596,2020-03-08,0.0000,0,1,city,MG,1,0
627859,NaN,31,2020-03-08,202011,21292666,21168791,False,False,1,0.00470,2020-03-08,0.0000,0,1,state,MG,1,0
627860,Divinópolis,3122306,2020-03-09,202011,240408,238230,False,False,1,0.41596,2020-03-09,0.0000,0,2,city,MG,0,0
627861,NaN,31,2020-03-09,202011,21292666,21168791,False,False,1,0.00470,2020-03-09,0.0000,0,2,state,MG,0,0
627862,Divinópolis,3122306,2020-03-10,202011,240408,238230,False,False,1,0.41596,2020-03-10,0.0000,0,3,city,MG,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1005150,Água Boa,3100609,2021-08-08,202132,13523,13735,False,True,511,3778.74732,2021-06-18,0.0137,7,408,city,MG,0,0
1005151,Água Comprida,3100708,2021-08-08,202132,1992,1999,False,True,169,8483.93574,2021-06-18,0.0473,8,436,city,MG,0,0
1005152,Águas Formosas,3100906,2021-08-08,202132,19247,19207,False,True,899,4670.85780,2021-06-18,0.0345,31,431,city,MG,0,0
1005153,Águas Vermelhas,3101003,2021-08-08,202132,13599,13539,False,True,402,2956.09971,2021-06-18,0.0174,7,446,city,MG,0,0


Escolhendo uma cidade específica para conhecer melhor algumas informações do dataset.

In [ ]:
cidade = "Florestal"
dadosCidade = dadosMG[dadosMG[0] == cidade]
display(dadosCidade)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
638502,Florestal,3126000,2020-05-25,202022,7533,7461,False,False,1,13.27492,2020-05-25,0.0000,0,1,city,MG,1,0
638905,Florestal,3126000,2020-05-26,202022,7533,7461,False,False,1,13.27492,2020-05-26,0.0000,0,2,city,MG,0,0
639318,Florestal,3126000,2020-05-27,202022,7533,7461,False,False,1,13.27492,2020-05-27,0.0000,0,3,city,MG,0,0
639744,Florestal,3126000,2020-05-28,202022,7533,7461,False,False,2,26.54985,2020-05-28,0.0000,0,4,city,MG,1,0
640182,Florestal,3126000,2020-05-29,202022,7533,7461,False,False,2,26.54985,2020-05-29,0.0000,0,5,city,MG,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1001165,Florestal,3126000,2021-08-04,202131,7533,7461,False,True,288,3823.17802,2021-06-18,0.0417,12,437,city,MG,0,0
1002020,Florestal,3126000,2021-08-05,202131,7533,7461,False,True,288,3823.17802,2021-06-18,0.0417,12,438,city,MG,0,0
1002875,Florestal,3126000,2021-08-06,202131,7533,7461,False,True,288,3823.17802,2021-06-18,0.0417,12,439,city,MG,0,0
1003730,Florestal,3126000,2021-08-07,202131,7533,7461,False,True,288,3823.17802,2021-06-18,0.0417,12,440,city,MG,0,0


In [ ]:
dadosCidade[16] = pd.to_numeric(dadosCidade[16].copy())
dadosCidade[17] = pd.to_numeric(dadosCidade[17].copy())
TotalCasos = dadosCidade[16].sum()
TotalObitos = dadosCidade[17].sum()
#Fonte: https://www.delftstack.com/pt/howto/python-pandas/how-to-get-the-sum-of-pandas-column/
print(f"Total de casos em {cidade} é {TotalCasos}")
print(f"Total de óbitos em {cidade} é {TotalObitos}")

Total de casos em Florestal é 288
Total de óbitos em Florestal é 12
